In [1]:
import glob
import pathlib
import pickle
import numpy as np
import sys
import os
from music21 import *
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split

sys.path.append("..")
from musiclearn import config

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Num GPUs Available:  1


In [2]:
# path to Schubert collection
schubert_dir = pathlib.Path(config.MUSICNET_MIDI_DIR) / "Schubert"

# path to final fantasy collection
ff_dir = pathlib.Path(config.FF_MIDI_DIR)

In [3]:
# length of an input sequence
SEQUENCE_LENGTH = 100

In [4]:
def get_notes(midi_dir):
    """ Get all the notes and chords from the midi files in the collection /ff_midi_songs """
    notes = []
    
    noteFile_dir = str(midi_dir) + "/notes"
    
    if not os.path.exists(noteFile_dir):
        files_dir = str(midi_dir) + "/*.mid"
        for file in glob.glob(files_dir):
            midi = converter.parse(file)

            print("Parsing %s" % file)

            notes_to_parse = None

            try: # file has instrument parts
                s2 = instrument.partitionByInstrument(midi)
                notes_to_parse = s2.parts[0].recurse()
            except: # file has notes in a flat structure
                notes_to_parse = midi.flat.notes

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                     notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))

        # save the features
        with open(noteFile_dir, 'wb') as fp:
            pickle.dump(notes, fp)
    else:
        with open(noteFile_dir, 'rb') as fp:
            notes = pickle.load(fp)
    
    return notes

In [5]:
# extract the notes
notes = get_notes(ff_dir)

In [6]:
# get the number of unique notes
n_vocab = len(set(notes))
print("No. unique notes =", n_vocab)

No. unique notes = 308


In [7]:
# get all pitch name
pitchNames = sorted(set(item for item in notes))

# create a dictionary to map notes to unique integers
note_to_int = dict((note, number) for number, note in enumerate(pitchNames))

# create a dictionary to map unique integers to notes
int_to_note = dict((number, note) for number, note in enumerate(pitchNames))

In [8]:
def prepare_sequences(sequence_length, notes, n_vocab, note_to_int):
    """ Prepare the sequences used by the neural network """   
    network_input = []
    network_output = []
    
    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i : i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
        
    n_samples = len(network_input)
    
    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_samples, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)
    
    network_output = np.array(network_output) #keras.utils.to_categorical(network_output)
    
    return (network_input, network_output)

In [9]:
def lstm_model(input_shape, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(input_shape[1], input_shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop')
    
    return model

In [10]:
def train_lstm_model(sequence_length, notes, n_vocab, note_to_int, epochs = 20, batch_size = 128):
    """ Train a neural network to generate music """
    # get amount of pitch names
    n_vocab = len(set(notes))
    
    network_input, network_output = prepare_sequences(sequence_length, notes, n_vocab, note_to_int)

    model = lstm_model(network_input.shape, n_vocab)
    
    # train the network
    filepath = "saved_weight.hdf5"
    
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        node='min'
    )
    
    callback_list = [checkpoint_cb]
    
    model.fit(network_input, network_output, epochs=epochs, batch_size=batch_size, callbacks=callback_list)

In [11]:
# test code to output model architecture
network_input, network_output = prepare_sequences(SEQUENCE_LENGTH, notes, n_vocab, note_to_int)
model = lstm_model(network_input.shape, n_vocab)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 512)          1052672   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
activation (Activation)      (None, 256)               0

In [12]:
# train the model
train_lstm_model(SEQUENCE_LENGTH, notes, n_vocab, note_to_int, 100, 128)

Epoch 1/100
350/350 [==============================] - 93s 255ms/step - loss: 5.4432
Epoch 2/100
350/350 [==============================] - 89s 255ms/step - loss: 4.6746
Epoch 3/100
350/350 [==============================] - 90s 256ms/step - loss: 4.7543
Epoch 4/100
350/350 [==============================] - 90s 256ms/step - loss: 4.5641
Epoch 5/100
350/350 [==============================] - 89s 254ms/step - loss: 4.5015
Epoch 6/100
350/350 [==============================] - 89s 253ms/step - loss: 4.4817
Epoch 7/100
350/350 [==============================] - 88s 253ms/step - loss: 4.4565
Epoch 8/100
350/350 [==============================] - 88s 252ms/step - loss: 4.4354
Epoch 9/100
350/350 [==============================] - 88s 250ms/step - loss: 4.4322
Epoch 10/100
350/350 [==============================] - 88s 251ms/step - loss: 4.4062
Epoch 11/100
350/350 [==============================] - 88s 250ms/step - loss: 4.3768
Epoch 12/100
350/350 [==============================] - 88s 250

350/350 [==============================] - 87s 248ms/step - loss: 0.4865
Epoch 93/100
350/350 [==============================] - 87s 248ms/step - loss: 0.4615
Epoch 94/100
350/350 [==============================] - 87s 248ms/step - loss: 0.4569
Epoch 95/100
350/350 [==============================] - 87s 248ms/step - loss: 0.4484
Epoch 96/100
350/350 [==============================] - 87s 248ms/step - loss: 0.4386
Epoch 97/100
350/350 [==============================] - 87s 248ms/step - loss: 0.4149
Epoch 98/100
350/350 [==============================] - 87s 248ms/step - loss: 0.4115
Epoch 99/100
350/350 [==============================] - 87s 248ms/step - loss: 0.3956
Epoch 100/100
350/350 [==============================] - 87s 248ms/step - loss: 0.3989


In [13]:
def prepare_sequences_to_predict(sequence_length, notes, n_vocab, note_to_int):
    """ Prepare the sequences to make prediction """
    network_input = []
    
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
    
    n_samples = len(network_input)
    
    # reshapre the input
    normalized_input = np.reshape(network_input, (n_samples, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)
    
    return (network_input, normalized_input)

In [14]:
def load_lstm_model(input_shape, n_vocab):
    """ Load the saved LSTM model """
    
    model = tf.keras.models.load_model('saved_weight.hdf5')
    
    return model

In [15]:
def generate_notes(model, network_input, int_to_note, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:]

    return prediction_output

In [16]:
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp=filename)

In [17]:
# generate a new midi file from the train model
noteFile_dir = str(ff_dir / "notes")
    
# load the notes used to train the model
with open(noteFile_dir, 'rb') as fp:
    notes = pickle.load(fp)

network_input, normalized_input = prepare_sequences_to_predict(SEQUENCE_LENGTH, notes, n_vocab, note_to_int)
    
# load saved weights
model = load_lstm_model(normalized_input.shape, n_vocab)
prediction_output = generate_notes(model, network_input, int_to_note, n_vocab)

 # make midi file
create_midi(prediction_output, "test_output_ff.mid")

## WaveNet model

In [ ]:
K.clear_session()

In [ ]:
def simplified_wavenet(n_vocab, sequence_length):
    """ A simplified version of WaveNet without residual and skip connection """
    model = Sequential()
    
    # embedding layer
    model.add(Embedding(n_vocab, 100, input_length=sequence_length, trainable=True))
    
    # causal convolution
    model.add(Conv1D(64, 3, padding='causal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPool1D(2))
    
    # dialated causal convolution
    model.add(Conv1D(128,3,activation='relu',dilation_rate=2,padding='causal'))
    model.add(Dropout(0.2))
    model.add(MaxPool1D(2))

    model.add(Conv1D(256,3,activation='relu',dilation_rate=4,padding='causal'))
    model.add(Dropout(0.2))
    model.add(MaxPool1D(2))
    
    #model.add(Conv1D(256,5,activation='relu'))    
    model.add(GlobalMaxPool1D())
    
    model.add(Dense(256, activation='relu'))
    model.add(Dense(n_vocab, activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
    
    return model

In [ ]:
wavenet_model = simplified_wavenet(n_vocab, SEQUENCE_LENGTH)
wavenet_model.summary()

In [ ]:
def prepare_sequences_wavenet(sequence_length, notes, n_vocab, note_to_int):
    """ Prepare the sequences used by the neural network """   
    network_input = []
    network_output = []
    
    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i : i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
        
    n_samples = len(network_input)
    
    # no reshape
    network_input = np.array(network_input)
    # normalize input
    network_input = network_input / float(n_vocab)
    
    network_output = np.array(network_output)
    
    return (network_input, network_output)

In [ ]:
# prepare the input to the model
X, Y = prepare_sequences_wavenet(SEQUENCE_LENGTH, notes, n_vocab, note_to_int)
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

In [ ]:
checkpoint_cb = ModelCheckpoint(
    "wavenet_weights_2.hdf5", 
    monitor='val_loss', 
    mode='min',
    save_best_only=True,
    verbose=1
    )

history = wavenet_model.fit(X_train,
                    Y_train,
                    batch_size=128,
                    epochs=100,
                    validation_data=(X_val, Y_val),
                    verbose=1,
                    callbacks=[checkpoint_cb]
                   )

In [ ]:
# load the trained model
model = tf.keras.models.load_model('wavenet_weights_2.hdf5')

In [ ]:
def generate_notes(model, network_input, int_to_note, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)

    pattern = list(network_input[start])
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern)))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:]

    return prediction_output

In [ ]:
# generate new sequence with wavenet
predicted_notes = generate_notes(wavenet_model, X_val, int_to_note, n_vocab)

# convert to MiDi file
create_midi(predicted_notes, "wavenet_sample_output.mid")